<div style="text-align: center; border: 4px solid #4285F4; border-radius: 15px; padding: 20px; margin-bottom: 25px; background-color: #e8f0fe;">
    <h1 style="color: #4285F4; margin:0;">📚 Audiobook Creator Studio (Google AI Edition) 🎙️</h1>
    <p style="font-size: 1.1em;"><b>Welcome to the definitive tool for converting your e-books into high-quality audiobooks using Google's powerful AI.</b></p>
</div>

---

### ✨ Key Features in this Studio Version:

**1. Integrated API Key Management:**
   - **No More `.env` Files:** Enter and save your Google AI API key directly in the new `Configuration` panel. The key is masked for security and stored for future sessions.

**2. Intelligent & Dynamic UI:**
   - **Context-Aware Options:** The UI adapts to your choices. For example, the 'Narrator' gender option automatically hides when you select the 'Multi-Voice' mode.
   - **E-Pub Cover Preview:** Automatically displays the book cover from `.epub` files for visual confirmation.

**3. Robust File & Process Management:**
   - **Safe Deletion:** Confirmation prompts now include the filename, preventing accidental deletions.
   - **Active Feedback:** The 'Generate' button shows a spinner, and a real-time progress bar and color-coded log keep you informed of the status.
   - **Cancellable Generations:** Stop a lengthy generation process at any time with the 'Cancel' button.

**4. Advanced Audio Controls:**
   - **Fine-Tuning:** A collapsible 'Advanced Options' section lets you control audio bitrate and inter-paragraph silence for professional-grade output.

---

### Instructions for Use:

1.  **Run the Setup Cell:** Execute the first code cell below. It will clone the necessary repository and install all required packages for Google AI.
2.  **Run the App Cell:** Execute the second code cell to launch the interactive UI.
3.  **Configure API Key:** In the UI, go to the `0. Configuration` tab, enter your Google AI API key, and click 'Save'. You can get one from the <a href="https://aistudio.google.com/app/apikey" target="_blank">Google AI Studio</a>.
4.  **Create Your Audiobook:** Follow the steps in the UI to select a book, configure your options, and generate!
<div style="border: 2px solid #fbbc05; background-color: #fffbeb; padding: 10px; margin-top: 20px; border-radius: 8px;">
    <p style="margin: 0;"><b>Note:</b> This notebook has been updated to use the <strong>Google AI API</strong>. The underlying generation logic in the cloned repository, which originally used OpenAI, would need to be rewritten to call Google's Text-to-Speech API. The UI has been fully adapted for this purpose.</p>
</div>

In [ ]:
%%capture
# This magic command hides the detailed output of the installation process.
# Remove '%%capture' if you need to debug the setup.

import os
import sys
import subprocess
import nest_asyncio
import asyncio
import io

# --- 1. CLONE REPOSITORY (if needed) ---
repo_dir = "audiobook-creator"
if not os.path.exists(repo_dir):
    print(f"Cloning repository...", flush=True)
    try:
        subprocess.check_call(["git", "clone", "https://github.com/BChrisUCC/audiobook-creator", repo_dir, "--depth", "1", "--quiet"])
        print("Repository cloned successfully.", flush=True)
    except Exception as e:
        print(f"ERROR: Failed to clone repository. Please ensure git is installed. Details: {e}", flush=True)
else:
    print(f"Repository '{repo_dir}' already exists.", flush=True)

# --- 2. ADD TO PATH & INSTALL PACKAGES ---
if os.path.isdir(repo_dir):
    sys.path.insert(0, os.path.abspath(repo_dir))
    # Swapped OpenAI with Google's packages
    required_packages = ["google-generativeai", "google-cloud-texttospeech", "tqdm", "pydub", "word2number", "python-dotenv", "nest_asyncio", "ipywidgets", "ebooklib", "Pillow"]
    print("Checking and installing required packages...", flush=True)
    subprocess.check_call([sys.executable, "-m", "pip", "install", *required_packages, "--quiet"])
    print("All packages are ready.", flush=True)
else:
    print(f"ERROR: Directory '{repo_dir}' not found. Cannot proceed.", flush=True)

# --- 3. CORE IMPORTS ---
import ipywidgets as widgets
from IPython.display import display, Audio, FileLink
from dotenv import load_dotenv
from PIL import Image
import ebooklib
from ebooklib import epub

# Import the main generation function from the cloned repo
# NOTE: This function would need to be modified to use Google's API instead of OpenAI's.
# We are providing a dummy function as a fallback to demonstrate the UI.
try:
    from generate_audiobook import process_audiobook_generation
    print("Successfully imported 'process_audiobook_generation'.", flush=True)
    print("WARNING: The imported function may still use OpenAI. A full conversion requires modifying that file.", flush=True)
except ImportError as e:
    print(f"Could not import from the cloned repository. Details: {e}", flush=True)
    # Define a dummy function to prevent crashes if the import fails
    async def process_audiobook_generation(*args, **kwargs):
        yield "<p style='color:orange;'><b>NOTE:</b> Using a placeholder generation function.</p>"
        yield "Progress: 10% - Starting up..."
        await asyncio.sleep(2)
        yield "Progress: 50% - Simulating audio generation..."
        await asyncio.sleep(2)
        yield "Progress: 100% - Done!"
        yield "<p style='color:green;'><b>SUCCESS:</b> Audiobook generation simulation completed.</p>"

# Apply nest_asyncio to allow nested event loops in Jupyter/Colab
nest_asyncio.apply()

# Load environment variables from the repo's .env file (if it exists)
load_dotenv(dotenv_path=os.path.join(repo_dir, '.env'))

print("\n✅ Setup complete. You can now run the next cell to launch the UI.")

In [ ]:
from types import SimpleNamespace

class AudiobookCreatorUI:
    """Manages the state and components of the Audiobook Creator UI."""
    def __init__(self, repo_dir="audiobook-creator"):
        # --- Configuration & State ---
        self.repo_dir = repo_dir
        self.book_dir = os.path.join(self.repo_dir, 'books')
        self.audio_dir = 'generated_audiobooks'
        os.makedirs(self.book_dir, exist_ok=True)
        os.makedirs(self.audio_dir, exist_ok=True)

        self.is_generating = False
        self.delete_book_pending = False
        self.delete_audio_pending = False
        
        # --- Build UI ---
        self._create_widgets()
        self._create_layout()
        self._register_handlers()
        self.initialize_ui_state()

    def _create_widgets(self):
        """Initializes all ipywidgets used in the UI."""
        style = {'description_width': 'initial'}
        layout_button = widgets.Layout(width='auto', margin='0 5px')
        
        # --- Header ---
        self.header = widgets.HTML('<h1></h1>') # Placeholder, set in layout
        
        # --- Panel 0: Configuration ---
        self.api_key_input = widgets.Password(description='Google AI API Key:', style=style, layout=widgets.Layout(width='400px'))
        self.save_api_key_button = widgets.Button(description='Save Key', button_style='primary', icon='save', layout=layout_button)
        self.api_key_status = widgets.HTML()

        # --- Panel 1: Source & Cover ---
        book_files = self._get_local_files(self.book_dir, ('.epub', '.pdf', '.txt'))
        self.book_dropdown = widgets.Dropdown(options=book_files, description='Select Book:', style=style, layout=widgets.Layout(flex='1'))
        self.upload_widget = widgets.FileUpload(accept='.epub,.pdf,.txt', multiple=False, description='Or Upload', layout=widgets.Layout(width='180px'))
        self.delete_book_button = widgets.Button(description='Delete Book', button_style='danger', icon='trash', layout=layout_button)
        self.delete_book_confirm = widgets.HTML()
        self.cover_preview = widgets.Image(format='png', width=200, height=300, layout=widgets.Layout(border='1px solid lightgray', margin='10px auto', object_fit='contain'))

        # --- Panel 2: Audio Config (Updated for Google TTS) ---
        self.tts_model = widgets.Dropdown(options=['Standard', 'WaveNet', 'Neural2'], value='WaveNet', description='Voice Technology:', style=style)
        self.voice_option = widgets.ToggleButtons(options=['Single Voice', 'Multi-Voice'], description='Voice Mode:', style=style)
        self.output_format = widgets.Dropdown(options=['M4B (Chapters & Cover)', 'mp3', 'aac', 'm4a', 'wav', 'opus', 'flac'], value='M4B (Chapters & Cover)', description='Format:', style=style)
        self.narrator_gender = widgets.ToggleButtons(options=['male', 'female'], description='Narrator:', style=style)
        self.advanced_toggle = widgets.Checkbox(value=False, description='Show Advanced Options', indent=False)
        self.bitrate_slider = widgets.Dropdown(options=['96k', '128k', '192k', '256k', '320k'], value='128k', description='Audio Bitrate:', style=style)
        self.silence_slider = widgets.FloatSlider(value=0.5, min=0, max=2.0, step=0.1, description='Inter-paragraph Silence (s):', readout_format='.1f', style=style, layout=widgets.Layout(width='95%'))
        
        # --- Panel 3: Generation & Logs ---
        self.generate_button = widgets.Button(description='Generate Audiobook', button_style='success', icon='cogs', layout=widgets.Layout(width='200px'))
        self.cancel_button = widgets.Button(description='Cancel', button_style='danger', icon='stop', disabled=True, layout=layout_button)
        self.progress_bar = widgets.IntProgress(value=0, min=0, max=100, description='Ready.', bar_style='success', orientation='horizontal', layout=widgets.Layout(margin='5px 0'))
        self.status_notification = widgets.HTML()
        self.log_output_html = widgets.HTML(layout={'border': '1px solid #ccc', 'height': '300px', 'overflow_y': 'auto', 'padding': '8px', 'background_color': '#f9f9f9', 'font-family': 'monospace'})

        # --- Panel 4: Review & Download ---
        self.audio_select = widgets.Dropdown(description='Generated File:', style=style, layout=widgets.Layout(flex='1'))
        self.play_button = widgets.Button(description='Play', button_style='info', icon='play', layout=layout_button)
        self.download_button = widgets.Button(description='Download', button_style='primary', icon='download', layout=layout_button)
        self.delete_audio_button = widgets.Button(description='Delete', button_style='danger', icon='trash', layout=layout_button)
        self.delete_audio_confirm = widgets.HTML()
        self.audio_output = widgets.Output(layout=widgets.Layout(margin_top='10px'))

        # Group widgets to be disabled during generation
        self.config_widgets = [self.api_key_input, self.save_api_key_button, self.book_dropdown, self.upload_widget, self.delete_book_button, self.tts_model, 
                               self.voice_option, self.output_format, self.narrator_gender, self.advanced_toggle, self.bitrate_slider, self.silence_slider]

    def _create_layout(self):
        """Assembles widgets into the final UI layout using an accordion."""
        pane_layout = widgets.Layout(padding='10px', border='1px solid transparent')
        
        # --- Define Panes ---
        api_key_info = widgets.HTML("Enter your Google AI API key below. You can get one from the <a href='https://aistudio.google.com/app/apikey' target='_blank'>Google AI Studio</a>. The key will be saved to a local `.env` file for future use.")
        pane0 = widgets.VBox([api_key_info, widgets.HBox([self.api_key_input, self.save_api_key_button]), self.api_key_status], layout=pane_layout)

        pane1 = widgets.VBox([widgets.HBox([self.book_dropdown, self.upload_widget]), widgets.HBox([self.delete_book_button, self.delete_book_confirm]), self.cover_preview], layout=pane_layout)

        advanced_options_box = widgets.VBox([self.bitrate_slider, self.silence_slider], layout=widgets.Layout(display='none', border='1px solid #ddd', padding='10px', margin_top='10px'))
        # Removed 'add_emotion_tags' widget
        pane2 = widgets.VBox([self.tts_model, self.voice_option, self.narrator_gender, self.output_format, widgets.HTML("<hr style='margin: 10px 0;'>"), self.advanced_toggle, advanced_options_box], layout=pane_layout)
        
        pane3 = widgets.VBox([widgets.HBox([self.generate_button, self.cancel_button]), self.progress_bar, self.status_notification, self.log_output_html], layout=pane_layout)
        
        pane4 = widgets.VBox([widgets.HBox([self.audio_select, self.play_button, self.download_button, self.delete_audio_button]), self.delete_audio_confirm, self.audio_output], layout=pane_layout)

        # --- Assemble Accordion ---
        self.accordion = widgets.Accordion(children=[pane0, pane1, pane2, pane3, pane4])
        self.accordion.set_title(0, '0. Configuration')
        self.accordion.set_title(1, '1. Select Source Book')
        self.accordion.set_title(2, '2. Configure Audio Settings')
        self.accordion.set_title(3, '3. Generate & Monitor')
        self.accordion.set_title(4, '4. Review & Download')

        # --- Final UI Assembly ---
        title_html = "<div style='text-align: center; border: 4px solid #4285F4; border-radius: 15px; padding: 10px; margin-bottom: 25px; background-color: #e8f0fe;'><h1 style='color: #4285F4; margin:0;'>📚 Audiobook Creator Studio (Google AI) 🎙️</h1></div>"
        self.full_ui = widgets.VBox([widgets.HTML(title_html), self.accordion])

    def _register_handlers(self):
        """Registers event handlers for widgets."""
        self.save_api_key_button.on_click(self._on_save_api_key_clicked)
        self.upload_widget.observe(self._handle_upload, names='value')
        self.book_dropdown.observe(self._update_cover_preview, names='value')
        self.delete_book_button.on_click(self._on_delete_book_clicked)
        self.voice_option.observe(self._on_voice_mode_change, names='value')
        self.advanced_toggle.observe(lambda change: self.accordion.children[2].children[-1].layout.display.set('block' if change.new else 'none'), names='value')
        self.generate_button.on_click(self._on_generate_clicked)
        self.cancel_button.on_click(self._on_cancel_clicked)
        self.play_button.on_click(self._on_play_clicked)
        self.download_button.on_click(self._on_download_clicked)
        self.delete_audio_button.on_click(self._on_delete_audio_clicked)
    
    # --- Helper & Utility Methods ---
    def _get_local_files(self, directory, extensions): return sorted([f for f in os.listdir(directory) if f.endswith(extensions)])

    def _log_message(self, message, level='info'):
        colors = {'info': '#333', 'success': 'green', 'warning': 'orange', 'error': 'red'}
        self.log_output_html.value += f"<p style='color:{colors.get(level, 'black')}; margin:2px;'>{message}</p>"

    def _set_ui_locked(self, is_locked):
        self.is_generating = is_locked
        for widget in self.config_widgets: widget.disabled = is_locked
        self.generate_button.disabled = is_locked
        self.cancel_button.disabled = not is_locked
        self.generate_button.icon = 'spinner fa-spin' if is_locked else 'cogs'
        
    def _update_file_lists(self):
        self.book_dropdown.options = self._get_local_files(self.book_dir, ('.epub', '.pdf', '.txt'))
        audio_files = self._get_local_files(self.audio_dir, ('.m4b', '.mp3', '.m4a', '.aac', '.wav', '.opus', '.flac'))
        self.audio_select.options = audio_files
        if audio_files: self.audio_select.value = audio_files[-1] # Default to the most recent

    # --- Event Handlers ---
    def _on_save_api_key_clicked(self, b):
        key = self.api_key_input.value
        if key:
            env_path = os.path.join(self.repo_dir, '.env')
            # Use GOOGLE_API_KEY for clarity
            with open(env_path, 'w') as f: f.write(f'GOOGLE_API_KEY={key}')
            load_dotenv(dotenv_path=env_path, override=True)
            self.api_key_status.value = "<b style='color:green;'>✓ API Key saved successfully!</b>"
        else:
            self.api_key_status.value = "<b style='color:red;'>✗ Please enter a valid API Key.</b>"

    def _handle_upload(self, change):
        if not change.new: return
        filename, fileinfo = list(change.new.items())[0]
        with open(os.path.join(self.book_dir, filename), 'wb') as f: f.write(fileinfo['content'])
        self._update_file_lists()
        self.book_dropdown.value = filename
        self.upload_widget.value.clear()

    def _update_cover_preview(self, change):
        book_path = os.path.join(self.book_dir, change.new or '')
        if book_path and book_path.endswith('.epub') and os.path.exists(book_path):
            try:
                cover_item = next(epub.read_epub(book_path).get_items_of_type(ebooklib.ITEM_COVER), None)
                if cover_item: self.cover_preview.value = cover_item.get_content(); return
            except Exception: pass
        img = Image.new('RGB', (200, 300), color='#f0f0f0'); b = io.BytesIO(); img.save(b, 'png'); self.cover_preview.value = b.getvalue()

    def _on_delete_book_clicked(self, b):
        book_to_delete = self.book_dropdown.value
        if not self.delete_book_pending:
            if book_to_delete: self.delete_book_confirm.value = f"<b style='color:red;'>Delete '{book_to_delete}'? Click again to confirm.</b>"
            self.delete_book_pending = True
        else:
            if book_to_delete: os.remove(os.path.join(self.book_dir, book_to_delete)); self._update_file_lists()
            self.delete_book_confirm.value = ""; self.delete_book_pending = False
            
    def _on_delete_audio_clicked(self, b):
        audio_to_delete = self.audio_select.value
        if not self.delete_audio_pending:
            if audio_to_delete: self.delete_audio_confirm.value = f"<b style='color:red;'>Delete '{audio_to_delete}'? Click again to confirm.</b>"
            self.delete_audio_pending = True
        else:
            if audio_to_delete: os.remove(os.path.join(self.audio_dir, audio_to_delete)); self._update_file_lists()
            self.delete_audio_confirm.value = ""; self.delete_audio_pending = False

    def _on_voice_mode_change(self, change):
        is_single_voice = change.new == 'Single Voice'
        self.narrator_gender.layout.display = 'flex' if is_single_voice else 'none'

    def _on_generate_clicked(self, b):
        if not os.environ.get("GOOGLE_API_KEY"):
            self.status_notification.value = "<b style='color:red;'>ERROR: Google AI API Key not found. Please set it in the 'Configuration' tab.</b>"
            self.accordion.selected_index = 0
            return
        if not self.book_dropdown.value:
            self.status_notification.value = "<b style='color:red;'>ERROR: Please select a book to generate.</b>"
            self.accordion.selected_index = 1
            return
        
        self._set_ui_locked(True)
        self.accordion.selected_index = 3
        self.log_output_html.value = ""
        self.progress_bar.value = 0
        self.status_notification.value = ""
        self._log_message(f"🚀 Starting generation for: {self.book_dropdown.value}")
        
        asyncio.create_task(self._run_generation_task())

    async def _run_generation_task(self):
        try:
            params = {
                'voice_option': self.voice_option.value, 'narrator_gender': self.narrator_gender.value,
                'output_format': self.output_format.value, 'book_path': os.path.join(self.book_dir, self.book_dropdown.value),
                'tts_model': self.tts_model.value, # Changed from add_emotion_tags
                'bitrate': self.bitrate_slider.value,
                'silence_duration': self.silence_slider.value
            }
            async for update in process_audiobook_generation(**params):
                if not self.is_generating: self._log_message("🛑 Generation cancelled by user.", 'warning'); break
                
                level = 'info'
                if "error" in update.lower(): level = 'error'
                elif "success" in update.lower() or "completed" in update.lower(): level = 'success'
                self._log_message(update, level)
                
                if "Progress:" in update:
                    try:
                        progress_val = int(float(update.split("Progress:")[1].split("%")[0]))
                        self.progress_bar.value = progress_val
                        self.progress_bar.description = update.split(" - ")[0].strip()
                    except (ValueError, IndexError): pass
                
                if "successfully created" in update or "completed" in update.lower():
                    self.status_notification.value = f"<b style='color:green;'>✅ {update}</b>"
                    self._update_file_lists()
                    self.accordion.selected_index = 4

        except Exception as e:
            self._log_message(f"FATAL ERROR: {e}", 'error')
            self.status_notification.value = f"<b style='color:red;'>An unexpected error occurred: {e}</b>"
        finally:
            self._set_ui_locked(False)
            self.progress_bar.description = "Finished."
            
    def _on_cancel_clicked(self, b): self.is_generating = False

    def _on_play_clicked(self, b):
        self.audio_output.clear_output(wait=True)
        with self.audio_output: 
            if self.audio_select.value: display(Audio(os.path.join(self.audio_dir, self.audio_select.value)))

    def _on_download_clicked(self, b):
        self.audio_output.clear_output(wait=True)
        with self.audio_output:
            if self.audio_select.value: display(FileLink(os.path.join(self.audio_dir, self.audio_select.value)))

    # --- Main Methods ---
    def initialize_ui_state(self):
        """Resets the UI to its default state."""
        self._update_file_lists()
        if self.book_dropdown.options: self.book_dropdown.value = self.book_dropdown.options[0]
        else: self._update_cover_preview(SimpleNamespace(new=None))
        
        self.api_key_input.value = ''
        self.api_key_status.value = "<b style='color:orange;'>API Key not set.</b>" if not os.environ.get("GOOGLE_API_KEY") else "<b style='color:green;'>✓ API Key is loaded.</b>"
        self.accordion.selected_index = 0 if not os.environ.get("GOOGLE_API_KEY") else 1
        self.status_notification.value = "<i>Welcome! Please configure your Google AI API key if this is your first time.</i>"
        self.log_output_html.value = ""
        self.progress_bar.value = 0
        self.progress_bar.description = "Ready."

    def display(self):
        """Renders the UI in the notebook."""
        display(self.full_ui)


In [ ]:
# Create an instance of the UI class and display it.
app = AudiobookCreatorUI()
app.display()